In [5]:
# google drive 연동
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [19]:
import os, pandas as pd
os.chdir('/content/drive/MyDrive/jiwon')

weather = pd.read_csv('해운대 날씨3.csv')
customer = pd.read_csv('해운대 입장객수3.csv')

In [20]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.layers import Bidirectional
from tensorflow.keras import optimizers
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import math
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.preprocessing import MaxAbsScaler,RobustScaler
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint
import datetime
import time
import inspect

# 날짜를 date type으로 변경 후, 나머지는 numeric type으로 변경
weather['날짜'] = pd.to_datetime(weather['날짜'], infer_datetime_format=True)
weather.iloc[:,1:] = weather.iloc[:,1:].apply(pd.to_numeric)

customer['방문일'] = pd.to_datetime(customer['방문일'], infer_datetime_format=True)
customer['방문객수'] = customer['방문객수'].str.replace(",","")
customer.iloc[:,1:] = customer.iloc[:,1:].apply(pd.to_numeric)

In [21]:
# 데이터 합치기
total_data = pd.merge(weather, customer, left_on='날짜', right_on="방문일", how='inner')
total_data = total_data[['강수_관측값', "기온", "습도", "체감온도", "평균수온", "평균풍속", "평균기압", "평균최대파고", "평균파주기", "방문객수"]]

In [22]:
# 결측치 평균, 0 채우기
total_data.loc[total_data['평균수온'] != total_data['평균수온'], '평균수온'] = total_data['평균수온'].mean()
total_data.loc[total_data['평균풍속'] != total_data['평균풍속'], '평균풍속'] = total_data['평균풍속'].mean()
total_data.loc[total_data['평균기압'] != total_data['평균기압'], '평균기압'] = total_data['평균기압'].mean()
total_data.loc[total_data['평균최대파고'] != total_data['평균최대파고'], '평균최대파고'] = total_data['평균최대파고'].mean()
total_data.loc[total_data['평균파주기'] != total_data['평균파주기'], '평균파주기'] = total_data['평균파주기'].mean()
total_data.loc[total_data['방문객수'] != total_data['방문객수'], '방문객수'] = total_data['방문객수'].mean()


## 방문객 수 예측

In [10]:
# input, output 지정
cos_x = total_data[['강수_관측값',"기온", "습도", "체감온도",  "평균수온", "평균풍속", "평균기압", "평균최대파고", "평균파주기","방문객수"]]
cos_y = total_data[['방문객수']]

In [11]:
# 방문객 수 예측을 위한 train/test split
train_x, test_x, train_y, test_y = train_test_split(cos_x,cos_y, test_size=0.2,random_state=42)

In [12]:
# 방문객 수 예측을 위한 scaler
cos_scaler =MinMaxScaler()
train_x = cos_scaler.fit_transform(train_x)
test_x = cos_scaler.transform(test_x)

In [14]:
train_x.shape,test_x.shape #((1175, 10), (294, 10))

((1175, 10), (294, 10))

In [15]:
train_x=train_x.reshape(1175,2,5)
test_x=test_x.reshape(294,2,5)

In [17]:
train_x.shape,test_x.shape #((1175, 2, 5), (294, 2, 5))

((1175, 2, 5), (294, 2, 5))

In [18]:
# 2. 모델구성 / 방문객
model=Sequential()
model.add(LSTM(units=128,input_shape=(2,5),return_sequences=True))
model.add(Bidirectional(LSTM(256)))
model.add(Dense(128, activation='swish'))
model.add(Dense(64, activation='swish'))
model.add(Dense(32, activation='swish'))
model.add(Dense(16, activation='swish'))
model.add(Dense(1))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 2, 128)            68608     
                                                                 
 bidirectional (Bidirectiona  (None, 512)              788480    
 l)                                                              
                                                                 
 dense (Dense)               (None, 128)               65664     
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                                 
 dense_3 (Dense)             (None, 16)                528       
                                                        

In [ ]:
# 3. 컴파일, 훈련 / 방문객
earlyStopping=EarlyStopping(monitor='val_loss',patience=300,mode='min',verbose=1,restore_best_weights=True) 
model.compile(loss='mse',optimizer='adam')
hist=model.fit(train_x,train_y,epochs=3000,batch_size=16,validation_split=0.2, callbacks=[earlyStopping])

Epoch 1/3000
59/59 [==============================] - 7s 31ms/step - loss: 29522907136.0000 - val_loss: 31081111552.0000
Epoch 2/3000
59/59 [==============================] - 1s 13ms/step - loss: 26165700608.0000 - val_loss: 19397191680.0000
Epoch 3/3000
59/59 [==============================] - 1s 13ms/step - loss: 15444505600.0000 - val_loss: 15323153408.0000
Epoch 4/3000
59/59 [==============================] - 1s 13ms/step - loss: 13876123648.0000 - val_loss: 13105574912.0000
Epoch 5/3000
59/59 [==============================] - 1s 13ms/step - loss: 8579992064.0000 - val_loss: 4037606400.0000
Epoch 6/3000
59/59 [==============================] - 1s 13ms/step - loss: 1383799296.0000 - val_loss: 521330464.0000
Epoch 7/3000
59/59 [==============================] - 1s 13ms/step - loss: 348365536.0000 - val_loss: 333097952.0000
Epoch 8/3000
59/59 [==============================] - 1s 21ms/step - loss: 240498240.0000 - val_loss: 246091408.0000
Epoch 9/3000
59/59 [=========================

In [ ]:
# 4. 평가, 예측 / 방문객
loss=model.evaluate(test_x,test_y)
pred_y=model.predict(test_x)

print('loss: ',loss)
print('예상 입장객 수: ', pred_y[-1:])

10/10 [==============================] - 0s 5ms/step - loss: 581496.3125
loss:  581496.3125
예상 입장객 수:  [[7947.9355]]


loss:  748262.125
예상 입장객 수:  [[8061.491]]

loss:  827985.6875
예상 입장객 수:  [[8287.672]]

## 날씨_예측

In [23]:
# input data
weather_input = total_data[['강수_관측값',"기온", "습도", "체감온도", "평균수온", "평균풍속", "평균기압", "평균최대파고", "평균파주기"]]
weather_output = total_data[['강수_관측값','기온']]

In [24]:
# train/test split
train_x, test_x, train_y, test_y = train_test_split(weather_input, weather_output, test_size=0.2, random_state=42)

In [25]:
# minmax scaler
min_scaler = MinMaxScaler()
train_x_scaled = min_scaler.fit_transform(train_x)
test_x_scaled = min_scaler.transform(test_x)

In [26]:
train_x_scaled.shape,test_x_scaled.shape #((1175, 9), (294, 9))

((1175, 9), (294, 9))

In [27]:
train_x = train_x_scaled.astype(float)
test_x = test_x_scaled.astype(float)
train_y = train_y.astype(float)
test_y = test_y.astype(float)

In [28]:
train_x = train_x_scaled.reshape(1175, 3, 3)
test_x = test_x_scaled.reshape(294, 3, 3)

In [29]:
train_x.shape, test_x.shape

((1175, 3, 3), (294, 3, 3))

In [ ]:
# 2.모델구성 / 날씨
model=Sequential()
model.add(LSTM(units=256,input_shape=(3,3),return_sequences=True))
model.add(Bidirectional(LSTM(512)))
model.add(Dense(256,activation='swish'))
model.add(Dense(128,activation='swish'))
model.add(Dense(64,activation='swish'))
model.add(Dense(32,activation='swish'))
model.add(Dense(16,activation='swish'))
model.add(Dense(2))
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 3, 256)            266240    
                                                                 
 bidirectional_2 (Bidirectio  (None, 1024)             3149824   
 nal)                                                            
                                                                 
 dense_10 (Dense)            (None, 256)               262400    
                                                                 
 dense_11 (Dense)            (None, 128)               32896     
                                                                 
 dense_12 (Dense)            (None, 64)                8256      
                                                                 
 dense_13 (Dense)            (None, 32)                2080      
                                                      

In [ ]:
# 3. 컴파일 훈련 / 날씨
earlyStopping = EarlyStopping(monitor='val_loss', patience=100, mode='min', verbose=1,restore_best_weights=True)
optimizer = optimizers.Adam(learning_rate=0.001)
model.compile(loss='mean_squared_error', optimizer=optimizer)
hist = model.fit(train_x, train_y, epochs=1000, batch_size=1024, 
                validation_split=0.2,
                callbacks = [earlyStopping],
                verbose=2)

Epoch 1/1000
1/1 - 10s - loss: 0.0489 - val_loss: 9.5111 - 10s/epoch - 10s/step
Epoch 2/1000
1/1 - 2s - loss: 8.4619 - val_loss: 65.9561 - 2s/epoch - 2s/step
Epoch 3/1000
1/1 - 2s - loss: 67.6590 - val_loss: 1.9190 - 2s/epoch - 2s/step
Epoch 4/1000
1/1 - 2s - loss: 1.6497 - val_loss: 22.3367 - 2s/epoch - 2s/step
Epoch 5/1000
1/1 - 2s - loss: 20.8638 - val_loss: 41.2903 - 2s/epoch - 2s/step
Epoch 6/1000
1/1 - 2s - loss: 39.0054 - val_loss: 43.4166 - 2s/epoch - 2s/step
Epoch 7/1000
1/1 - 2s - loss: 40.1044 - val_loss: 33.1987 - 2s/epoch - 2s/step
Epoch 8/1000
1/1 - 2s - loss: 29.0310 - val_loss: 21.0622 - 2s/epoch - 2s/step
Epoch 9/1000
1/1 - 2s - loss: 17.6248 - val_loss: 15.0839 - 2s/epoch - 2s/step
Epoch 10/1000
1/1 - 2s - loss: 14.5903 - val_loss: 15.2215 - 2s/epoch - 2s/step
Epoch 11/1000
1/1 - 2s - loss: 17.9082 - val_loss: 15.4916 - 2s/epoch - 2s/step
Epoch 12/1000
1/1 - 2s - loss: 19.7988 - val_loss: 12.7635 - 2s/epoch - 2s/step
Epoch 13/1000
1/1 - 2s - loss: 16.4079 - val_loss: 

In [ ]:
# 4. 평가, 예측 / 날씨
loss=model.evaluate(test_x,test_y)
pred_y=model.predict(test_x)

print('loss: ',loss)
print('예상 강수량, 기온 : ', pred_y[-1:])

10/10 [==============================] - 0s 30ms/step - loss: 0.0459
loss:  0.04593612998723984
예상 강수량, 기온 :  [[ 0.0800702 22.211409 ]]
